## Onchain-Coin-Analysis with Dexscreener API

Given a list of coin tickers ($MYCOIN) or coin names this Jupyter Notebook uses the Dexsreener API (https://docs.dexscreener.com/api/reference) to get onchain cryptocurrency-data. It filters and orders the coin-data in a way that might provide an initial overview over potential short-term investment opportunities.

As Dexscreener does not have an API endpoint to get a list of available coins (yet!), the temporary solution is to manually copy the coin-list from https://dexscreener.com/{my_chain} to a txt-file (*dxs-out*). Websrapping did not work for me unfortunately.

I have tested this with the Solana-Blockchain so far, but it should work for any Blockchain featured on dexscreener.com

In [242]:
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [302]:
df = pd.read_fwf("dxs-out")

df = df.set_axis(['data'], axis=1)

# Find the indices where the value is '/'
indices = df.index[df['data'] == '/'].tolist()

# Get the indices of the rows just before each '/'
prev_indices = [idx - 1 for idx in indices if idx > 0]

# Subset the dataframe to include only these rows
subset_df = df.loc[prev_indices]

tickers = subset_df['data'].tolist()

In [313]:
def search_pairs(queries):
    """
    Searches dexscreener for token pairs.

    :param queries: List of coin tickers from Dexscreener.
    :return: Dict of coin-data.
    """
    results = {}
    for query in queries:
        url = f"https://api.dexscreener.com/latest/dex/search/?q={query}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if len(data.get('pairs', [])) > 1: # if multiple pairs for a ticker exist
                data["multi-pairs"] = 1
                results[query] = data
            else:
                data["multi-pairs"] = 0
                results[query] = data
        else:
            results[query] = {"error": f"Failed to retrieve data: {response.status_code}"}
            
    return results

def filter_results(results, target_chain = 'solana'):
    """
    Reduces dexscreener output dict size leaving only useful output.

    :param results: dexscreener output dict.
    :param target_chain: string target blockchain (defaul = solana)
    :return: DataFrame with filtered dexscreener output (unique pairs only).
    """
    filtered_results = {}
    
    def get_age(created_at): #get age of coin from created_at UNIX timestamp
        current_time = datetime.now()
        created_timestamp = created_at / 1000  # Convert from milliseconds to seconds
        created_datetime = datetime.fromtimestamp(created_timestamp)
        delta = current_time - created_datetime
        age = round((delta.total_seconds() / 3600),2)
        return age

    def handle_multi_pairs(data, target_chain = 'solana'): 
        #if a ticker has multiple pairs, choose pair with highest 24h volume on the target chain
        highest = 0
        for d in data["pairs"]:
            if d["volume"]['h24'] > float(highest):
                if target_chain in d['chainId']:
                    best_pair = d
                    
            highest = d["volume"]['h24']     
        return best_pair

    def filter_data(pair):
        # some requests return non-complete data
        if 'fdv' not in pair:
            pair['fdv'] = np.nan

        if 'pairCreatedAt' not in pair:
            pair['pairCreatedAt'] = 1
            
        filtered_data = {
        'name': pair['baseToken']['name'],
        'symbol': pair['baseToken']['symbol'],
        'pairAddress': pair['pairAddress'],
        'txns_h24': (pair['txns']['h24']['buys'] + pair['txns']['h24']['buys']), # sum buys and sells
        'volume_h24': pair['volume']['h24'],
        'priceChange_h1': pair['priceChange']['h1'],
        'priceChange_h6': pair['priceChange']['h6'],
        'priceChange_h24': pair['priceChange']['h24'],
        'liquidity_usd': pair['liquidity']['usd'],
        'fdv': pair['fdv'],
        'age_hours': get_age(pair['pairCreatedAt'])
        }
        return filtered_data
        
    for query, data in results.items():
        if 'pairs' in data:
            if len(data['pairs']) == 1:
                pair = data['pairs'][0]
                filtered_results[query] = filter_data(pair)

            
            elif len(data['pairs']) > 1:
                pair = handle_multi_pairs(data)
                filtered_results[query] = filter_data(pair)
            
        else:
            filtered_results[query]= data # Retains error messages or multiple pairs data

    df = pd.DataFrame.from_dict(filtered_results, orient='index')
    df["tstamp"] = datetime.now()
        
    return df
    

def filter_liquidity(df, liquidity = 15000):
    """
    Reduces the coin-dataframe: removes low liquidity coins.

    :param df: DataFrame with filtered dexscreener output.
    :param liquidity: Min Threshold for liquidity in pool in USD (default = 15k)
    :return: Subsampled DataFrame with liquid pairs only.
    """    
    return df[df['liquidity_usd'] > liquidity].copy()        


def vol_fdv_reorder(df):
    """
    Reorders the coin-dataframe: df ordered by vol/fdv.

    :param df: DataFrame with dexscreener output.
    :return: Reorded DataFrame.
    """    
    
    df.loc[:, "vol/fdv"] = df["volume_h24"] / df["fdv"]

    df = df.sort_values(by = ["vol/fdv"], ascending = False)

    return df

In [304]:
queries = ["mylist"]
raw = search_pairs(tickers) # raw data from dexscreener API

In [314]:
df = filter_results(raw, target_chain = 'solana') # reduced data, multi-pairs handling
liquid = filter_liquidity(df,liquidity = 15000) # filter for liquid pairs
out = vol_fdv_reorder(liquid) # order by vol/fdv ratio

In [316]:
out = out.drop(columns=['priceChange_h1', 'priceChange_h6', 'priceChange_h24',"name"])
out

,symbol,pairAddress,txns_h24,volume_h24,liquidity_usd,fdv,age_hours,tstamp,vol/fdv
LONG,SOLONG,AoJmvcu4d5J4Krzm6F6ch9LCX76NsXyS4UeU4cDytfPU,4174,337479.63,17633.74,121172,15.92,2023-12-25 11:56:44.381209,2.785129
KOWALSKI,KOWALSKI,t359zKmHDcwXFgRRjTzXhzveAwWisPD7ZxknK2pf9KA,3012,294265.56,59169.16,168544,12.19,2023-12-25 11:56:44.381209,1.745927
YLLIS,YLLIS,4mX7jKEFLXsF9LTmtwJYUossN14P7UMqhEBFVgVrY858,1378,229976.86,48023.02,389603,82.71,2023-12-25 11:56:44.381209,0.590285
ADA,ADA,CdhFPj7hFShKVjznSshCVqbapJctLuL2izxQnMx2PztM,51424,708991.51,228980.16,1652532,244.92,2023-12-25 11:56:44.381209,0.429033
HO! HO! HO,GIF,HgGYu5xThtcQ8tMKLZVsydqfsMBef6A6z8NSQaaKwSUG,1388,194744.99,102869.93,521501,161.64,2023-12-25 11:56:44.381209,0.373432
SONLY UP,POPDOG,FmLmM2C3svyMmRwzTmnEjGnA5cD4YBLuPoQPixqZJG15,448,44138.47,21718.63,118208,139.49,2023-12-25 11:56:44.381209,0.373397
KPOP,KPOP,1482U6Rhfgfm3sdrSH1sCnoH2fzkXowD9Rv4JshWdoYq,1888,581013.17,309624.72,2216116,251.59,2023-12-25 11:56:44.381209,0.262176
PEPE,PEPE,4YcwqxR7eoH29rF2G7U9YmtzCt1n7tyHcywg3Emif85s,264,23638.48,40603.52,150681,8108.44,2023-12-25 11:56:44.381209,0.156878
pepe,PEPE,4YcwqxR7eoH29rF2G7U9YmtzCt1n7tyHcywg3Emif85s,264,23638.48,40603.52,150681,8108.44,2023-12-25 11:56:44.381209,0.156878
OMNI,OMNI,FB4vuLH6KJpvEvzZrFPgdyEVFdYFzsdTN5wxY8RqFPfV,5426,1854959.29,112865.44,15184162,59.39,2023-12-25 11:56:44.381209,0.122164
